In [1]:
from tqdm import tqdm
from collections import Counter
import random
from copy import copy
import numpy as np
from dataclasses import dataclass, field

In [41]:
%%bash
FILE_NAME="knapsack1.txt"
LINK="https://d3c33hcgiwev3.cloudfront.net/_6dfda29c18c77fd14511ba8964c2e265_knapsack1.txt?Expires=1618617600&Signature=LZlPVnblJ~m~8Q7mToEGyC0fMvoZSyxfNdA2RPRYwGEYX4Hxwbo-dKEGkJVeM~nGHwN0Z5z-YYccwNXq6LJbWRz60GXATrn~xAi5ujpOL2URtO6PRxMkQNUyVhDYuyzxDT-tAUE0A5q~KNiuG3LxA91LwEdXX8hDfx5Ld~iPXQk_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"

wget -O $FILE_NAME $LINK 

--2021-04-16 17:35:26--  https://d3c33hcgiwev3.cloudfront.net/_6dfda29c18c77fd14511ba8964c2e265_knapsack1.txt?Expires=1618617600&Signature=LZlPVnblJ~m~8Q7mToEGyC0fMvoZSyxfNdA2RPRYwGEYX4Hxwbo-dKEGkJVeM~nGHwN0Z5z-YYccwNXq6LJbWRz60GXATrn~xAi5ujpOL2URtO6PRxMkQNUyVhDYuyzxDT-tAUE0A5q~KNiuG3LxA91LwEdXX8hDfx5Ld~iPXQk_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A
Resolving d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)... 52.85.114.45, 52.85.114.74, 52.85.114.40, ...
Connecting to d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)|52.85.114.45|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 987 [text/plain]
Saving to: ‘knapsack1.txt’

     0K                                                       100%  134M=0s

2021-04-16 17:35:26 (134 MB/s) - ‘knapsack1.txt’ saved [987/987]



In [23]:
%%bash
FILE_NAME="knapsack_big.txt"
LINK="https://d3c33hcgiwev3.cloudfront.net/_6dfda29c18c77fd14511ba8964c2e265_knapsack_big.txt?Expires=1618617600&Signature=CHGw6NNzSdAF6~2s5CXS0GvCnePN4Vnjqv9kuTsFff10m5mtX2WTzklcf0iH2KeSHcM~yUNPcMB-7bbe4xEbQb6s5v~KnTtZVrFfXlqII6ZqkHGLxQBg05mWK8l7yitAUiZqWViPvjzG~Y27jYor8aAzimpNwBugDU5AjSP5U8Y_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A"

wget -O $FILE_NAME $LINK 

--2021-04-16 03:59:07--  https://d3c33hcgiwev3.cloudfront.net/_6dfda29c18c77fd14511ba8964c2e265_knapsack_big.txt?Expires=1618617600&Signature=CHGw6NNzSdAF6~2s5CXS0GvCnePN4Vnjqv9kuTsFff10m5mtX2WTzklcf0iH2KeSHcM~yUNPcMB-7bbe4xEbQb6s5v~KnTtZVrFfXlqII6ZqkHGLxQBg05mWK8l7yitAUiZqWViPvjzG~Y27jYor8aAzimpNwBugDU5AjSP5U8Y_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A
Resolving d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)... 52.85.114.40, 52.85.114.45, 52.85.114.74, ...
Connecting to d3c33hcgiwev3.cloudfront.net (d3c33hcgiwev3.cloudfront.net)|52.85.114.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25570 (25K) [text/plain]
Saving to: ‘knapsack_big.txt’

     0K .......... .......... ....                            100% 22,6M=0,001s

2021-04-16 03:59:08 (22,6 MB/s) - ‘knapsack_big.txt’ saved [25570/25570]



In [42]:
! ls -lh

total 120
-rw-r--r--  1 Alexander  staff    27K Apr 16 17:32 knapsack.ipynb
-rw-r--r--  1 Alexander  staff   987B Mar 29  2016 knapsack1.txt
-rw-r--r--  1 Alexander  staff    25K Mar 29  2016 knapsack_big.txt


In [45]:
from dataclasses import dataclass
@dataclass
class Item:
    size: int
    value: int

# Knapsack

In [60]:
FILE_NAME="knapsack1.txt"

In [61]:
with open(FILE_NAME) as file:
    a = file.read().splitlines()

In [62]:
knapsack_size, number_of_items = (int(x) for x in a[0].split(' '))
knapsack_size, number_of_items

(10000, 100)

In [63]:
a[1:10]

['16808 250',
 '50074 659',
 '8931 273',
 '27545 879',
 '77924 710',
 '64441 166',
 '84493 43',
 '7988 504',
 '82328 730']

In [64]:
items = [ tuple(map(int, x.split(' '))) for x in a[1:] ]
items = [ Item(size=x[1], value=x[0]) for x in items ]

In [65]:
items[:3]

[Item(size=250, value=16808),
 Item(size=659, value=50074),
 Item(size=273, value=8931)]

In [73]:
import numpy as np

def reconstruct_knapsack(A, items):
    item_num, budget = A.shape
    item_num -= 1
    budget -= 1
    choosen_items = []
    while budget > 0 and item_num > 0:
        if A[item_num, budget] == A[item_num-1, budget]:
            item_num -= 1
        else:
            choosen_items.append(item_num)
            budget -= items[item_num-1].size
            item_num -= 1
    return choosen_items
            

def knapsack(budget, items):
    n = len(items)
    A = np.zeros((n+1, budget+1), dtype=np.int)
    A[0, :] = 0
    for i in range(1, n+1):
        new_item = items[i-1]
        for j in range(1, budget+1):
            if new_item.size > j:
                A[i, j] = A[i-1, j]
            else:
                A[i, j] = max(A[i-1, j], 
                              A[i-1, j-new_item.size] + new_item.value)
    choosen_items = reconstruct_knapsack(A, items)
    return A, choosen_items

In [77]:
A, choosen_items = knapsack(knapsack_size, items)
A[-1, -1]

2493893

In [78]:
knapsack(budget=6, items = [ 
    Item(size=4, value=3),
    Item(size=3, value=2),
    Item(size=2, value=4),
    Item(size=3, value=4)
])

(array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 3, 3, 3],
        [0, 0, 0, 2, 3, 3, 3],
        [0, 0, 4, 4, 4, 6, 7],
        [0, 0, 4, 4, 4, 8, 8]]),
 [4, 3])

# Knapsack big

In [34]:
FILE_NAME="knapsack_big.txt"

In [35]:
with open(FILE_NAME) as file:
    a = file.read().splitlines()

knapsack_size, number_of_items = (int(x) for x in a[0].split(' '))
knapsack_size, number_of_items

(2000000, 2000)

In [36]:
items = [ tuple(map(int, x.split(' '))) for x in a[1:] ]
items = [ Item(value=x[0], size=x[1]) for x in items ]

In [37]:
items[:5]

[Item(value=16808, size=241486),
 Item(value=50074, size=834558),
 Item(value=8931, size=738037),
 Item(value=27545, size=212860),
 Item(value=77924, size=494349)]

In [38]:
import numpy as np

def rec(A, items, n_items, budget):
    if n_items < 0 or budget < 0:
        return 0
    if A[n_items, budget] != -1:
        return A[n_items, budget]
    
    new_item = items[n_items-1]
    a1 = rec(A, items, n_items-1, budget)
    if budget-new_item.size < 0:
        a2 = 0
    else:
        a2 = rec(A, items, n_items-1, budget-new_item.size) + new_item.value
    A[n_items, budget] = max(a1, a2)

    return A[n_items, budget]
    
    
def knapsack_big(budget, items):
    n = len(items)
    A = np.zeros((n+1, budget+1), dtype=np.int) - 1
    A[0, :] = 0
    return A, rec(A, items, n, budget)

In [39]:
%%time
A, v = knapsack_big(knapsack_size, items)
v

CPU times: user 50.7 s, sys: 27.2 s, total: 1min 17s
Wall time: 1min 22s


4243395

In [41]:
A

array([[      0,       0,       0, ...,       0,       0,       0],
       [     -1,      -1,      -1, ...,      -1,      -1,   16808],
       [     -1,      -1,      -1, ...,      -1,      -1,   66882],
       ...,
       [     -1,      -1,      -1, ...,      -1,      -1, 4243395],
       [     -1,      -1,      -1, ...,      -1,      -1, 4243395],
       [     -1,      -1,      -1, ...,      -1,      -1, 4243395]])

In [40]:
# knapsack_big(budget=6, items = [ 
#     Item(value=3, size=4),
#     Item(value=2, size=3),
#     Item(value=4, size=2),
#     Item(value=4, size=3)
# ])